## Model blending
### Import, Read data

In [1]:
#package load
from sklearn.model_selection import ParameterSampler
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import os
dicpath = os.getcwd()

In [23]:
lgbm_train01 = pd.read_csv('fin_train_01.csv',index_col=0)
lgbm_train02 = pd.read_csv('fin_train_02.csv',index_col=0).sort_values('acc_id')
lgbm_test01 = pd.read_csv('fin_test_01.csv',index_col=0)
lgbm_test02 = pd.read_csv('fin_test_02.csv',index_col=0).sort_values('acc_id').reset_index(drop=True)

- 이전에 전처리한 데이터들 불러오기

### modeling & blending

In [24]:
le = LabelEncoder()

lgbm_train01["label"] = le.fit_transform(lgbm_train01["label"])
lgbm_train02["label"] = le.fit_transform(lgbm_train02["label"])


#### lgbm_train01을 활용한 모델링

In [12]:
%%time

num_folds = 5

# Create arrays and dataframes to store results
feats1 = [f for f in lgbm_train01.columns if f not in ['label','acc_id']]
feats2 = [f for f in lgbm_train02.columns if f not in ['label','acc_id']]

folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)

result_e = []
preds= []

num_trials = 1
num_rounds = 3000
early_stopping_rounds = 200
params = {
    'objective':['multiclass'],
    'num_class':[4],
    'class_weight':['balanced'],
    'boosting':['gbdt'],
    #'min_child_weight': list(np.arange(1,20,1)),
    'colsample_bytree': [0.6],
    #'max_depth': list(np.arange(3,15,1)),
    #'subsample': list(np.arange(0.4,1,0.1))
    'reg_lambda': [10],
    'reg_alpha' : [1]
    #'learning_rate': [0.005,0.01,0.05,0.1],
    #'num_leaves' :list(np.arange(20,55,3)),
}
selected_params = []

for trial, param in zip(np.arange(num_trials),list(ParameterSampler(params, n_iter=num_trials))):
    result_b = []
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(lgbm_train01[feats1], lgbm_train01['label'])):
    
        #if n_fold != 0:
        #    break
        
        train_x, train_y = lgbm_train01[feats1].iloc[train_idx], lgbm_train01['label'].iloc[train_idx]
        valid_x, valid_y = lgbm_train01[feats1].iloc[valid_idx], lgbm_train01['label'].iloc[valid_idx] 
        

        lgb_train = lgb.Dataset(train_x, train_y)
        lgb_eval = lgb.Dataset(valid_x, valid_y, reference=lgb_train)
        
        regressor01 = lgb.train(param,
                              lgb_train,
                              valid_sets = lgb_eval,
                              num_boost_round = num_rounds,
                              verbose_eval=50,
                              early_stopping_rounds=early_stopping_rounds
                             )

        y_pred = regressor01.predict(valid_x)
        preds.append(y_pred)

        max_pred = []
        for i in range(len(y_pred)):
            max_pred.append(np.argmax(y_pred[i]))

        comp_lab = pd.DataFrame({'pred':max_pred,'true':valid_y})

        pr_rc = pd.DataFrame()
        for i in range(len(comp_lab['pred'].value_counts())):
            prec = len(comp_lab[(comp_lab["pred"]==i) & (comp_lab["true"]==i)])/len(comp_lab["pred"][comp_lab["pred"]==i])
            recl = len(comp_lab[(comp_lab["pred"]==i) & (comp_lab["true"]==i)])/len(comp_lab["true"][comp_lab["true"]==i])

            if i == 0:
                pr_rc = pd.DataFrame({"pred":[prec],"recl":[recl]})
            else:
                pr_rc = pr_rc.append(pd.DataFrame({"pred":[prec],"recl":[recl]}))

        f_score = list()
        for i in range(len(pr_rc.index)):
            f_score.append(1/(pr_rc["pred"].tolist()[i]))
            f_score.append(1/(pr_rc["recl"].tolist()[i]))
        ls_result_lgb = 8/sum(f_score)
        
        result_b.append(ls_result_lgb)
        
        
    selected_params.append(param)
    result_e.append(result_b)

Training until validation scores don't improve for 200 rounds.
[50]	valid_0's multi_logloss: 0.696154
[100]	valid_0's multi_logloss: 0.665882
[150]	valid_0's multi_logloss: 0.655467
[200]	valid_0's multi_logloss: 0.648391
[250]	valid_0's multi_logloss: 0.643693
[300]	valid_0's multi_logloss: 0.639974
[350]	valid_0's multi_logloss: 0.636634
[400]	valid_0's multi_logloss: 0.63401
[450]	valid_0's multi_logloss: 0.631272
[500]	valid_0's multi_logloss: 0.629455
[550]	valid_0's multi_logloss: 0.627964
[600]	valid_0's multi_logloss: 0.62675
[650]	valid_0's multi_logloss: 0.625836
[700]	valid_0's multi_logloss: 0.625018
[750]	valid_0's multi_logloss: 0.624578
[800]	valid_0's multi_logloss: 0.624126
[850]	valid_0's multi_logloss: 0.623588
[900]	valid_0's multi_logloss: 0.62323
[950]	valid_0's multi_logloss: 0.622696
[1000]	valid_0's multi_logloss: 0.622349
[1050]	valid_0's multi_logloss: 0.622093
[1100]	valid_0's multi_logloss: 0.622036
[1150]	valid_0's multi_logloss: 0.622004
[1200]	valid_0's 

#### lgbm_train02을 활용한 모델링

In [13]:
%%time
preds2 = []

for trial, param in zip(np.arange(num_trials),list(ParameterSampler(params, n_iter=num_trials))):
    result_b = []
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(lgbm_train02[feats2], lgbm_train02['label'])):
        
        #if n_fold != 0:
        #    break
        #model 02 light gbm with 306
        train_x, train_y = lgbm_train02[feats2].iloc[train_idx], lgbm_train02['label'].iloc[train_idx]
        valid_x, valid_y2 = lgbm_train02[feats2].iloc[valid_idx], lgbm_train02['label'].iloc[valid_idx] 


        lgb_train = lgb.Dataset(train_x, train_y)
        lgb_eval = lgb.Dataset(valid_x, valid_y2, reference=lgb_train)

        regressor02 = lgb.train(param,
                              lgb_train,
                              valid_sets = lgb_eval,
                              num_boost_round = num_rounds,
                              verbose_eval=100,
                              early_stopping_rounds=early_stopping_rounds
                             )
        
        y_pred = regressor02.predict(valid_x)
        preds2.append(y_pred)
        
        
        max_pred = []
        for i in range(len(y_pred)):
            max_pred.append(np.argmax(y_pred[i]))

        comp_lab = pd.DataFrame({'pred':max_pred,'true':valid_y2})

        pr_rc = pd.DataFrame()
        for i in range(len(comp_lab['pred'].value_counts())):
            prec = len(comp_lab[(comp_lab["pred"]==i) & (comp_lab["true"]==i)])/len(comp_lab["pred"][comp_lab["pred"]==i])
            recl = len(comp_lab[(comp_lab["pred"]==i) & (comp_lab["true"]==i)])/len(comp_lab["true"][comp_lab["true"]==i])

            if i == 0:
                pr_rc = pd.DataFrame({"pred":[prec],"recl":[recl]})
            else:
                pr_rc = pr_rc.append(pd.DataFrame({"pred":[prec],"recl":[recl]}))

        f_score = list()
        for i in range(len(pr_rc.index)):
            f_score.append(1/(pr_rc["pred"].tolist()[i]))
            f_score.append(1/(pr_rc["recl"].tolist()[i]))
        ls_result_lgb = 8/sum(f_score)
        
        result_b.append(ls_result_lgb)
        
        
    selected_params.append(param)
    result_e.append(result_b)

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's multi_logloss: 0.661005
[200]	valid_0's multi_logloss: 0.639819
[300]	valid_0's multi_logloss: 0.627973
[400]	valid_0's multi_logloss: 0.621113
[500]	valid_0's multi_logloss: 0.616033
[600]	valid_0's multi_logloss: 0.612186
[700]	valid_0's multi_logloss: 0.60964
[800]	valid_0's multi_logloss: 0.607602
[900]	valid_0's multi_logloss: 0.605954
[1000]	valid_0's multi_logloss: 0.604797
[1100]	valid_0's multi_logloss: 0.604165
[1200]	valid_0's multi_logloss: 0.604013
[1300]	valid_0's multi_logloss: 0.604075
[1400]	valid_0's multi_logloss: 0.604314
Early stopping, best iteration is:
[1272]	valid_0's multi_logloss: 0.603782
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's multi_logloss: 0.659816
[200]	valid_0's multi_logloss: 0.637685
[300]	valid_0's multi_logloss: 0.625512
[400]	valid_0's multi_logloss: 0.6176
[500]	valid_0's multi_logloss: 0.611607
[600]	valid_0's multi_logloss: 0.607

#### 5-fold cv 에서 각각 lgbm_train01/02 모델링 결과 

In [15]:
result_e

[[0.7160195098593874,
  0.7137735765609082,
  0.7143850862660739,
  0.7074211596942418,
  0.7044487240688693],
 [0.7210618279558735,
  0.7227018623512634,
  0.7236471857559741,
  0.7214727934597943,
  0.7166287761749196]]

#### 두 모델의 예측값(softmax function 결과)의 평균
- 검증셋에서의 결과로 대략적 비율을 정함

In [18]:
temp_01 = []
for i in enumerate(folds.split(lgbm_train01)):
    temp_01.append(i)

vaild_tmp1 = lgbm_train02['label'].iloc[temp_01[0][1][1]]
vaild_tmp2 = lgbm_train02['label'].iloc[temp_01[1][1][1]]
vaild_tmp3 = lgbm_train02['label'].iloc[temp_01[2][1][1]]
vaild_tmp4 = lgbm_train02['label'].iloc[temp_01[3][1][1]]
vaild_tmp5 = lgbm_train02['label'].iloc[temp_01[4][1][1]]

tmps = [vaild_tmp1,vaild_tmp2,vaild_tmp3,vaild_tmp4,vaild_tmp5]

for j in range(5): 
    a = [preds[j],preds2[j]]
    max_pred_avg =  []   
    b = np.average(a, axis=0).tolist()

    for i in range(len(b)):
        max_pred_avg.append(np.argmax(b[i]))  

    comp_lab = pd.DataFrame({'pred':max_pred_avg,'true':tmps[j]})
    comp_lab["pred"] = comp_lab["pred"].astype(int)
    print('pred:',sum(comp_lab["pred"]==0),sum(comp_lab["pred"]==1),sum(comp_lab["pred"]==2),sum(comp_lab["pred"]==3))
    print('true:',sum(comp_lab["true"]==0),sum(comp_lab["true"]==1),sum(comp_lab["true"]==2),sum(comp_lab["true"]==3))

    #comp_lab = pd.DataFrame({'pred':max_pred,'true':valid_y})

    pr_rc = pd.DataFrame()
    for i in range(len(comp_lab['pred'].value_counts())):
        prec = len(comp_lab[(comp_lab["pred"]==i) & (comp_lab["true"]==i)])/len(comp_lab["pred"][comp_lab["pred"]==i])
        recl = len(comp_lab[(comp_lab["pred"]==i) & (comp_lab["true"]==i)])/len(comp_lab["true"][comp_lab["true"]==i])
        print("precision : ",prec,", recall : ",recl)
        if i == 0:
            pr_rc = pd.DataFrame({"pred":[prec],"recl":[recl]})
        else:
            pr_rc = pr_rc.append(pd.DataFrame({"pred":[prec],"recl":[recl]}))

    f_score = list()
    for i in range(len(pr_rc.index)):
        f_score.append(1/(pr_rc["pred"].tolist()[i]))
        f_score.append(1/(pr_rc["recl"].tolist()[i]))
    ls_result_lgb = 8/sum(f_score)
    print(ls_result_lgb)
    print('------------------------------------')

pred: 5421 3863 5398 5318
true: 5031 4979 4966 5024
precision :  0.6342003320420586 , recall :  0.6833631484794276
precision :  0.7077400983691432 , recall :  0.5491062462341836
precision :  0.774546128195628 , recall :  0.8419250906161901
precision :  0.8544565626175253 , recall :  0.9044585987261147
0.7253584412252198
------------------------------------
pred: 5387 4009 5295 5309
true: 5076 4923 4970 5031
precision :  0.6441433079636161 , recall :  0.6836091410559496
precision :  0.6852082813669245 , recall :  0.5579930936420882
precision :  0.7852691218130312 , recall :  0.8366197183098592
precision :  0.8560934262572989 , recall :  0.9033989266547406
0.7265736716516179
------------------------------------
pred: 5646 3751 5281 5322
true: 4959 5023 4980 5038
precision :  0.6234502302515055 , recall :  0.7098205283323251
precision :  0.714476139696081 , recall :  0.5335456898267967
precision :  0.783753077068737 , recall :  0.8311244979919679
precision :  0.8526869597895528 , recall :

### Note

In [ ]:

'''
2018/08/09
feature engineering 과정이 한참 더 필요(EDA도 필요함)
    -시리얼데이터 어떻게 반영해햐니
    -노트에 반영된 아이디어 구현하기
    
multiclass를 어떻게 핸들링해야할지(loss function??)

2018/08/10
조금더 향상된 성능 & cv에서도 안정적으로 결과가 나옴
파라미터 서치는 따로 진행하지 않고 3fold cv를 learning_rate : 0.05, num_rounds : 2000 으로 진행

1순위 : multi_logloss를 loss function으로 사용하였는데, 
        01) 문제를 0.25, 0.5, 0.75, 1 형태로 레이블을 바꾼다음 logistic형태로 풀어볼까?
        02)
2순위 : feature을 경우 아직 train_party, train_trade를 사용하지 않음 + 노트에 정리된 것도 반영X



2018/08/13
1차 제출 : 0.6896 (23/46)
이전에 만든 코드를 test set에 적용해서 나온 결과
파라미터 서치는 따로 진행하지 않고 3fold cv를 learning_rate : 0.05, num_rounds : 2000 으로 진행

1순위 : multi_logloss를 loss function으로 사용하였는데, 
        01) 문제를 0.25, 0.5, 0.75, 1 형태로 레이블을 바꾼다음 logistic형태로 풀어볼까?
        02) multi_auroc 사용 -> multi_auroc code 작성하기
        
2순위 : feature을 경우 아직 train_party, train_trade를 사용하지 않음 + 노트에 정리된 것도 반영X
        01) 파생변수 생성
        02) 특성 선택 방법 ( 모델기반[feature importance]특성 선택, data_leakage 체크, 상관관계 체크)

3순위 : 결과의 threshhold??, hyper parameter??, 다른 모델 활용(xgboost, nn, lr 등등)

2018/08/14
[0.6947982139926345, 0.6901791726318248, 0.6818321058701344] <- 기존

[0.7252078093749054, 0.7369634579247871, 0.7334708110879647] <- 1순위 01)
실제 제출 결과(2차 제출)는 0.5644 threshhold의 문제로 보임 'week'만 15000개... 

[0.6727887000512414, 0.642219781355088, 0.6692332969310869] <- 1순위 02)
auroc의 경우 성능이 multi_logloss에 비해 하락. 나중에 시간이 되면 다시 코드에 이상이 없는지 체크 일단 패스

1순위 : multi_logloss를 loss function으로 사용하였는데, 
        01) 문제를 1,2,3,4 형태로 레이블을 바꾼다음 logistic형태로 풀어볼까?
        02) multi_auroc 사용 -> multi_auroc code 작성하기
        03) 다른 방법.. f1 해보기
        
2순위 : feature을 경우 아직 train_party, train_trade를 사용하지 않음 + 노트에 정리된 것도 반영X
        01) 파생변수 생성
        02) 특성 선택 방법 ( 모델기반[feature importance]특성 선택, data_leakage 체크, 상관관계 체크)

3순위 : 결과의 threshhold??, hyper parameter??, 다른 모델 활용(xgboost, nn, lr 등등)

2018/08/16
3차 제출 : 0.6967 (26/60)
- f1 score를 사용하였는데 제대로 업데이트가 안되는(0.5수준에서 시작하여 미미한 향상)것을 발견.. 어떤부분이 문제인지 파악 못함
    => 일단 multi-logloss를 사용해서 진행(1순위 변경)
- feature생성 계획
    생성 => 모델링 => feature importance 확인 => 가지치기 => 생성 => 모델링...
    현재 feature importanve에서는 예상대로 play time이 가장 주요한 변수로 나타남
    생성방법 : feature tools를 이용한 영혼없는 생성, 가설 설정과 그에 기반한 변수 생성

- issue 이메일 문의 결과
"""
'train_trade.csv' 에서 유저들의 거래 기록을 보고 있는데, 배경 지식에 기반하여 생각할 때 

거래는 소스유저(source_acc_id) 와 타겟유저(target_acc_id)간의 상호간의 작용이 대부분이라고 판단하였습니다.
(A유저가 ㄱ아이템을 B에게 주면 B유저는 그에 상응하는 재화나 ㄴ아이템을 주는 방식. 물론 일방향의 거래도 
발생할 수 있지만 가능성은 전자의 경우가 많을 것으로 판단)

그런데 거래 주, 일, 시간별로 sorting해서 살펴본 결과 상호간의 거래는 거의 없고 일방향으로 거래를 하는 것으로 보여져서 이에 대해 
1. 데이터가 샘플링된 것인지 2. 거래 시스템상 정상적으로 발생된 데이터인지 궁금하여 질문을 남깁니다.

----------------------------------------------------------------------------------------------------
<엔씨소프트  답변>
 
실제로 일방향의 거래가 대부분임.
추가로, 거래 데이터의 경우 추출 오류로인해 중복 집계된 데이터 존재함. 
이를 감안하여 분석 진행하기 바람.
----------------------------------------------------------------------------------------------------
a. 일방향 형태의 거래가 더 많다
b. 중복집계된 데이터가 존재

a') 순수하게 user to user로 기부? 형태의 거래가 많이 발생하는지, 현금 거래처럼 게임 외부에서의 거래 이후 거래가 발생하는지,
    경매장 시스템으로 인해 일방향의 거래만 집계가 되는지
        => 1:1 교환 건 수가 없기 때문에 1,2번 형태는 아닌거 같음... 실제로 인게임에서 어떻게 거래가 형성되고 있는지 알아보기
        위해 직접 육성 중...
        
b') 중복 집계된 데이터는 unique를 통해 제거할 필요가 있음(날짜+시간+아이템+수량정도로 묶어서 중복을 제거하면 될듯)
        => 제거완료(데이터 사이즈가 절반 가까이 소멸.. nc 당신은 대체...)

"""

- 30*(mean,max,min =3)여개의 신규 feature 생성
    activity 데이터 1차 완료

- 파티와 거래데이터 활용방법에 대한 아이디어 필요
    지하철에서 생각하기
    

2018/08/17
작업 구분
A : loss function, parameter search...
B : data EDA(feature importance check, data leakage, correlation...)
C : feature engineering
D : model selection, blending

오늘 주요 작업 B, C.
- activity 이외의 변수들로 변수 생성하기

- 상관관계 체크

아이디어
-비율로 feature를 생성하는 것 말고 다른 방식을 생각해보기
-table 간의 cross feature
-featuretools 에서 더 다양한 agg 방식을 적용해보기


2018/08/19
주요 작업 B, C
- trade, party변수에 대한 idea 정리하기 & 변수 생성

- 상관관계 확인 ( data leakage 체크는 할 필요 없는듯 <= 베이스 모델에서 cv평균이랑 리더보드 결과랑 차이 0.001 이하)

아이디어
-비율로 feature를 생성하는 것 말고 다른 방식을 생각해보기
-table 간의 cross feature
-featuretools 에서 더 다양한 agg 방식을 적용해보기

2018/08/20
주요 작업 B, C
- trade변수로는 파생변수 생성완료
- party는 인원이 1~60이상으로 다양하게 나옴 좀더 생각해보고 파생변수를 만들어야 할듯
- 수요일 이내로 1차 파생변수 생성은 마무리하고 best single model 찾기(BO) -> 시작은 lgb로.. 8월내로 1차 마무리
- 9월 첫주 : 블렌딩 or stacking 으로 최적의 조합찾기(BO활용)
- 9월 둘쨋주 : 버퍼

아이디어
-비율이 같은 경우라면 선형으로 예측한뒤 valid셋의 비율만큼 threshhold정하기??
-비율로 feature를 생성하는 것 말고 다른 방식을 생각해보기
-table 간의 cross feature
-featuretools 에서 더 다양한 agg 방식을 적용해보기

2018/08/23
B,C 작업 1단계 완료 -> 추후에 feature 더 생성할때 추가하기 이제 그만 
cv5fold 성능 평균 : 0.7073105337940665

상위 feature importance
play_time의 std
play_time의 평균
cnt_dt의 std
play_time의 max
cnt_dt의 평균


주요 작업 A
random search와 Bayesian optimization을 이용하고 비교해보기


할것
-피클 저장(all_ft_np,filt_ft_np)
-random search, Bayesian optimization
-neural net으로 모델만들기(dnn, rnn이 가능성있을듯?)

아이디어
-블렌딩에 쓸 feature들로 각 주의 값들(총 8개가 될 듯)을 사용??, weight도 시간별로 줄 수 있을듯?

2018/08/26

-Bayesian optimization에서 lossfunction customize 하는 법을 알지 못해... parameter search로 변경
(시간도 BO에서 약 19시간+가 걸림...)
-뉴럴넷에 쓸 수 있도록 파일 가공하기

2018/09/03
reg_lambda = 10
col.. = 0.8


2018/09/05
파생변수 생성중...
+dnn model도 돌리는즁

'''